In [33]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, assemble, Aer
from qiskit.visualization import plot_histogram
from math import pi

def circuitMaker(n_qubits, n_cbits, encodings, parameters):
    # create quantum circuit
    qr = QuantumRegister(n_qubits, 'qubit')
    cr = ClassicalRegister(n_cbits, 'cr')
    qc = QuantumCircuit()
    qc.add_register(qr)
    qc.add_register(cr)
    
    # encoding block
    encoder0, encoder1 = encoder(encodings)
    # print(encodings, encoder1)
    # qc.rx(pi, encoder0)
    # qc.rz(pi, encoder0)
    qc.rx(pi, encoder1)
    qc.rz(pi, encoder1)
    qc.barrier(qr)
    
    for i in range(n_qubits):
        # entangle block
        if i < n_qubits - 1:
            qc.cnot(i, i+1)
        else:
            qc.barrier(qr)

    for i in range(n_qubits):
        # variational block
        qc.u3(parameters[i][0], parameters[i][1], parameters[i][2], i)

    # measurement
    qc.measure(qr, cr)
    
    return qc

def encoder(encodings):
    return [i for i, b in enumerate(encodings) if b == '0'], [i for i, b in enumerate(encodings) if b == '1']


In [48]:
import numpy as np
circuit_size = 4
parameters = [[1,1,1],[1,1,1],[1,1,1],[1,1,1]]

def findAction(statistic):
    distribution = [0]*circuit_size
    for k, v in statistic.items():
        # attention: reserved statistic
        for i, c in enumerate(k[::-1]):
            if c == '1':
                distribution[i] += v
    print(distribution)
    return distribution

def getAction(state, parameters):
    b_state = f'{state:0{circuit_size}b}'
    qc = circuitMaker(circuit_size, circuit_size, b_state, parameters)
    aer_sim = Aer.get_backend('aer_simulator')
    qobj = assemble(qc, shots=1000)
    job = aer_sim.run(qobj)
    rlt = job.result().get_counts()
    # print(rlt)
    return findAction(rlt)

[243, 203, 773, 750]


2

In [ ]:
import gym
import random

MAXEPISODE = 10
MAXSTEPS = 10
MINIBATCH = 5
GAMMA = 0.9
db = {}

env = gym.make('FrozenLake-v1', map_name = '4x4')
env.reset()
env.render()

for i in range(MAXEPISODE):
    state = env.reset()
    for t in range(MAXSTEPS):
        # get action for next step from target network
        distribution_target = getAction(state, parameters)
        action = np.argmax(distribution_target)
        # new position, reward, condition and probabilities
        new_state, reward, done, info = env.step(action=action)
        print(f'act: {action}, i:{type(i)}, new state:{new_state}, reward:{reward}, done:{done}, info:{info}')
        # save data in database
        db[i, t] = (state, action, reward, new_state, done)
        # sample from database
        if len(db) > MINIBATCH:
            sampled_ks = random.sample(len(db), MINIBATCH)
            sampled_vs = [db[k] for k in sampled_ks]
            # gradient descent
            v_targets = [(s[2] if s[4] else s[2] + GAMMA*np.max([i/sum(distribution_target) for i in distribution_target])) for s in sampled_vs]
            dis_predicted = [getAction(s[0], parameters) for s in sampled_vs]
            v_predicteds = [i[action]/sum(dis_predicted) for i in dis_predicted]
            cost = sum([(tar - pred)**2 for tar, pred in zip(v_targets, v_predicteds)])/MINIBATCH
            
        if done:
            break


In [45]:
parameters = [[1,1,1],[1,1,1],[1,1,1],[1,1,1]]
circuit_size = 4
b_state = '1011'

qc = circuitMaker(circuit_size, circuit_size, b_state, parameters)
qc.draw()

0100


┌───────┐┌───────┐ ░                 ░ ┌───────────┐┌─┐         
qubit_0: ┤ Rx(π) ├┤ Rz(π) ├─░───■─────────────░─┤ U3(1,1,1) ├┤M├─────────
         └───────┘└───────┘ ░ ┌─┴─┐           ░ ├───────────┤└╥┘┌─┐      
qubit_1: ───────────────────░─┤ X ├──■────────░─┤ U3(1,1,1) ├─╫─┤M├──────
         ┌───────┐┌───────┐ ░ └───┘┌─┴─┐      ░ ├───────────┤ ║ └╥┘┌─┐   
qubit_2: ┤ Rx(π) ├┤ Rz(π) ├─░──────┤ X ├──■───░─┤ U3(1,1,1) ├─╫──╫─┤M├───
         ├───────┤├───────┤ ░      └───┘┌─┴─┐ ░ ├───────────┤ ║  ║ └╥┘┌─┐
qubit_3: ┤ Rx(π) ├┤ Rz(π) ├─░───────────┤ X ├─░─┤ U3(1,1,1) ├─╫──╫──╫─┤M├
         └───────┘└───────┘ ░           └───┘ ░ └───────────┘ ║  ║  ║ └╥┘
   cr: 4/═════════════════════════════════════════════════════╩══╩══╩══╩═
                                                              0  1  2  3